In [1]:
# from ResidualStrategy_simple_dynamic_record import DynamicResidualModelStrategy
# from residualstrategy_simple import DynamicResidualModelStrategy
from record import DynamicResidualModelStrategy
from datetime import datetime
from vnpy.app.portfolio_strategy import BacktestingEngine
from vnpy.trader.constant import Interval
import pandas as pd
from pyecharts.charts import (Bar,Line,Graph,Gauge,Page)#柱状图，折线图，关系图，仪表盘,多图同表
from pyecharts import options as opts
import numpy as np
import random
import talib

In [ ]:
engine = BacktestingEngine()
engine.clear_data()

engine.set_parameters(
    vt_symbols=["HC888.SHFE", 'RB888.SHFE'],
    interval=Interval.MINUTE,
    start=datetime(2014, 12, 31),
    end=datetime(2019, 5, 14),
    rates={"HC888.SHFE": 5/10000, "RB888.SHFE": 5/10000},
    slippages={"HC888.SHFE":2, "RB888.SHFE":1},
    sizes={"HC888.SHFE":10, "RB888.SHFE":10},
    priceticks={"HC888.SHFE":2, "RB888.SHFE":1},
    capital=1_000_0,
    collection_names={"HC888.SHFE":"HC888", "RB888.SHFE":"RB888"}

)
engine.add_strategy(DynamicResidualModelStrategy, {'price_add':20})
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
# 



2020-05-14 12:36:04.582504	开始加载历史数据
2020-05-14 12:36:05.320578	HC888.SHFE加载进度： [2%]
2020-05-14 12:36:05.913951	HC888.SHFE加载进度： [4%]
2020-05-14 12:36:06.683892	HC888.SHFE加载进度： [6%]
2020-05-14 12:36:07.449844	HC888.SHFE加载进度： [8%]
2020-05-14 12:36:08.236123	HC888.SHFE加载进度： [9%]
2020-05-14 12:36:08.955199	HC888.SHFE加载进度：# [11%]
2020-05-14 12:36:09.948543	HC888.SHFE加载进度：# [13%]
2020-05-14 12:36:10.879179	HC888.SHFE加载进度：# [15%]
2020-05-14 12:36:11.555337	HC888.SHFE加载进度：# [17%]
2020-05-14 12:36:12.199649	HC888.SHFE加载进度：# [19%]
2020-05-14 12:36:13.165098	HC888.SHFE加载进度：## [21%]
2020-05-14 12:36:14.066656	HC888.SHFE加载进度：## [23%]
2020-05-14 12:36:14.837732	HC888.SHFE加载进度：## [24%]
2020-05-14 12:36:15.418101	HC888.SHFE加载进度：## [26%]
2020-05-14 12:36:16.356375	HC888.SHFE加载进度：## [28%]
2020-05-14 12:36:17.173413	HC888.SHFE加载进度：### [30%]
2020-05-14 12:36:17.727965	HC888.SHFE加载进度：### [32%]
2020-05-14 12:36:18.405324	HC888.SHFE加载进度：### [34%]
2020-05-14 12:36:19.215300	HC888.SHFE加载进度：### [36%]
2020-05-14 

In [ ]:
record_array = engine.strategy.spread_record
datetime_array = engine.strategy.datetime_record

In [ ]:
# mean = talib.SMA(np.array(record_array),240*30).tolist()
mean = (talib.SMA(np.array(record_array),240*30))
std= np.sqrt(talib.VAR(np.array(record_array),240*30))
boll_up = list(mean + 3*std)
boll_down = list(mean - 3*std)


x = datetime_array
(
    Line()
    .add_xaxis(x)
    .add_yaxis(
        "上轨",
        boll_up
    )    .add_yaxis(
        "实际价差",
        y
    ) .add_yaxis(
        "下轨",
        boll_down
    )
    .set_global_opts(title_opts=opts.TitleOpts(title=""))
    .render_notebook())

In [ ]:
ind=-1
# 5年的实际价差
x, y = datetime_array[:ind], record_array[:ind]
(
    Line()
    .add_xaxis(x)
    .add_yaxis(
        "实际价差",
        y
    )
    .set_global_opts(title_opts=opts.TitleOpts(title=""))
    .render_notebook())

In [ ]:
same_direction_symbol= engine.strategy.y_symbol.split('.')[0]
inverse_direction_symbol= engine.strategy.x_symbol.split('.')[0]

datetime_list = []
direction_list = []
offset_list = []
volume_list = []
price_list = []
for order in engine.strategy.orders.values():

     
    if order.symbol == same_direction_symbol:
        datetime = order.datetime
        direction = order.direction.value
        offset = order.offset.value
        volume = order.volume
        price = order.price
        
        datetime_list.append(datetime)
        direction_list.append(direction)
        offset_list.append(offset)
        volume_list.append(volume)
        price_list.append(price)
        
same_direction_order_df = pd.DataFrame({'datetime':datetime_list,'direction':direction_list,'offset':offset_list,'volume':volume_list,'price':price_list})
same_direction_order_df['symbol'] = same_direction_symbol


In [ ]:
datetime_list = []
direction_list = []
offset_list = []
volume_list = []
price_list = []
for order in engine.strategy.orders.values():

     
    if order.symbol == inverse_direction_symbol:
        datetime = order.datetime
        direction = order.direction.value
        offset = order.offset.value
        volume = order.volume
        price = order.price
        
        datetime_list.append(datetime)
        direction_list.append(direction)
        offset_list.append(offset)
        volume_list.append(volume)
        price_list.append(price)
        
inverse_direction_order_df = pd.DataFrame({'datetime':datetime_list,'direction':direction_list,'offset':offset_list,'volume':volume_list,'price':price_list})
inverse_direction_order_df['symbol'] = same_direction_symbol

In [ ]:
combine_order_df = pd.merge(left=same_direction_order_df,right=inverse_direction_order_df,left_on=['datetime','offset'],right_on=['datetime','offset'],suffixes=('_y', '_x'))
combine_order_df

In [ ]:
orders_analyze_df

In [ ]:


if combine_order_df.shape[0] % 2 != 0 :
    
    order_num = (combine_order_df.shape[0] -1)
else:
    order_num = combine_order_df.shape[0]

start_ind = np.arange(0,order_num,2)
end_ind = start_ind + 1

orders_analyze_df = pd.DataFrame({},columns=['open_date','close_date','direction','open_spread','close_spread','y_profit','x_profit'])
for ind in np.arange(start_ind.shape[0]):   
    one_order_df = combine_order_df.iloc[start_ind[ind]:end_ind[ind]+1]
    if one_order_df.iloc[0,1] == '多':
        
        y_profit = one_order_df['price_y'].diff().iloc[-1]*one_order_df['volume_y'].iloc[0]
        x_profit = -one_order_df['price_x'].diff().iloc[-1]*one_order_df['volume_x'].iloc[0]
    
    else:
        y_profit = -one_order_df['price_y'].diff()*one_order_df['volume_y'].iloc[0]
        x_profit = one_order_df['price_x'].diff()*one_order_df['volume_x'].iloc[0]
    
    start_spread = one_order_df['price_y'].iloc[0]*one_order_df['volume_y'].iloc[0]-one_order_df['price_x'].iloc[0]*one_order_df['volume_x'].iloc[0]
    close_spread = one_order_df['price_y'].iloc[1]*one_order_df['volume_y'].iloc[1]-one_order_df['price_x'].iloc[1]*one_order_df['volume_x'].iloc[1]
    
    
    start_datetime = one_order_df['datetime'].iloc[0]
    end_datetime = one_order_df['datetime'].iloc[1]
    direction = one_order_df['direction_y'].iloc[0]
    orders_analyze_df = orders_analyze_df.append({'open_date':start_datetime,'close_date':end_datetime,
                                                  'direction':direction,
                                                  'open_spread':start_spread,
                                                  'close_spread':close_spread,
                                                  'y_profit':y_profit,
                                                  'x_profit':x_profit},ignore_index=True)
    


orders_analyze_df['total_profit'] = (orders_analyze_df['y_profit']+orders_analyze_df['x_profit'])*np.where(orders_analyze_df['direction']=='多',1,-1)

In [ ]:
one_order_df

In [ ]:
same_direction_symbol= engine.strategy.y_symbol.split('.')[0]
inverse_direction_symbol= engine.strategy.x_symbol.split('.')[0]

In [ ]:
datetime_list = []
direction_list = []
offset_list = []
volume_list = []
price_list = []
for trade in engine.trades.values():

    if trade.symbol == same_direction_symbol:
        datetime = trade.datetime
        direction = trade.direction.value
        offset = trade.offset.value
        volume = trade.volume
        price = trade.price
        
        datetime_list.append(datetime)
        direction_list.append(direction)
        offset_list.append(offset)
        volume_list.append(volume)
        price_list.append(price)
    same_direction_df = pd.DataFrame({'datetime':datetime_list,'direction':direction_list,'offset':offset_list,'volume':volume_list,'price':price_list})
    same_direction_df['symbol'] = same_direction_symbol

In [ ]:
datetime_list = []
direction_list = []
offset_list = []
volume_list = []
price_list = []
for trade in engine.trades.values():  
    if trade.symbol == inverse_direction_symbol:
        
        datetime = trade.datetime
        direction = trade.direction.value
        offset = trade.offset.value
        volume = trade.volume
        price = trade.price
       
        datetime_list.append(datetime)
        direction_list.append(direction)
        offset_list.append(offset)
        volume_list.append(volume)
        price_list.append(price)
        
    inverse_direction_df = pd.DataFrame({'datetime':datetime_list,'direction':direction_list,'offset':offset_list,'volume':volume_list,'price':price_list})
    inverse_direction_df['symbol'] = inverse_direction_symbol

In [ ]:
combine_trade_df = pd.merge(left=same_direction_df,right=inverse_direction_df,left_on=['datetime','offset'],right_on=['datetime','offset'],suffixes=('_y', '_x'))

if combine_trade_df.shape[0] % 2 != 0 :
    
    traded_num = (combine_trade_df.shape[0] -1)
else:
    traded_num = combine_trade_df.shape[0]

start_ind = np.arange(0,traded_num,2)
end_ind = start_ind + 1

trades_analyze_df = pd.DataFrame({},columns=['open_date','close_date','direction','open_spread','close_spread','y_profit','x_profit'])
for ind in np.arange(start_ind.shape[0]):   
    one_trade_df = combine_trade_df.iloc[start_ind[ind]:end_ind[ind]+1]
    if one_trade_df.iloc[0,1] == '多':
        
        y_profit = one_trade_df['price_y'].diff().iloc[-1]*one_trade_df['volume_y'].iloc[0]
        x_profit = -one_trade_df['price_x'].diff().iloc[-1]*one_trade_df['volume_x'].iloc[0]
    
    else:
        y_profit = -one_trade_df['price_y'].diff()*one_trade_df['volume_y'].iloc[0]
        x_profit = one_trade_df['price_x'].diff()*one_trade_df['volume_x'].iloc[0]
    start_spread = one_trade_df['price_y'].iloc[0]*one_trade_df['volume_y'].iloc[0]-one_trade_df['price_x'].iloc[0]*one_trade_df['volume_x'].iloc[0]
    close_spread = one_trade_df['price_y'].iloc[1]*one_trade_df['volume_y'].iloc[1]-one_trade_df['price_x'].iloc[1]*one_trade_df['volume_x'].iloc[1]
    start_datetime = one_trade_df['datetime'].iloc[0]
    end_datetime = one_trade_df['datetime'].iloc[1]
    direction = one_trade_df['direction_y'].iloc[0]
    trades_analyze_df = trades_analyze_df.append({'open_date':start_datetime,'close_date':end_datetime,
                                                  'direction':direction,
                                                  'open_spread':start_spread,
                                                  'close_spread':close_spread,
                                                  'y_profit':y_profit,
                                                  'x_profit':x_profit},ignore_index=True)
    


trades_analyze_df['total_profit'] = (trades_analyze_df['y_profit']+trades_analyze_df['x_profit'])*np.where(trades_analyze_df['direction']=='多',1,-1)

In [ ]:
trades_analyze_df

In [ ]:
combine_trade_df

In [ ]:
from residualstrategy_simple_with_moving_exit import DynamicResidualModelStrategy
engine = BacktestingEngine()

engine.set_parameters(
    vt_symbols=["HC888.SHFE", 'RB888.SHFE'],
    interval=Interval.MINUTE,
    start=datetime(2018, 12, 31),
    end=datetime(2019, 3, 30),
    rates={"HC888.SHFE": 5/10000, "RB888.SHFE": 5/10000},
    slippages={"HC888.SHFE":2, "RB888.SHFE":1},
    sizes={"HC888.SHFE":10, "RB888.SHFE":10},
    priceticks={"HC888.SHFE":2, "RB888.SHFE":1},
    capital=1_000_0,
    collection_names={"HC888.SHFE":"HC888", "RB888.SHFE":"RB888"}

)
engine.add_strategy(DynamicResidualModelStrategy, {})
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
# 

In [ ]:
combine_trade_df

In [ ]:
combine_trade_df = pd.merge(left=same_direction_df,right=inverse_direction_df,left_on=['datetime','offset'],right_on=['datetime','offset'],suffixes=('_y', '_x'))

if combine_trade_df.shape[0] % 2 != 0 :
    
    traded_num = (combine_trade_df.shape[0] -1)
else:
    traded_num = combine_trade_df.shape[0]

start_ind = np.arange(0,traded_num,2)
end_ind = start_ind + 1

trades_analyze_df = pd.DataFrame({},columns=['open_date','close_date','direction','open_spread','close_spread','y_profit','x_profit'])
for ind in np.arange(start_ind.shape[0]):   
    one_trade_df = combine_trade_df.iloc[start_ind[ind]:end_ind[ind]+1]
    if one_trade_df.iloc[0,1] == '多':
        
        y_profit = one_trade_df['price_y'].diff().iloc[-1]*one_trade_df['volume_y'].iloc[0]
        x_profit = -one_trade_df['price_x'].diff().iloc[-1]*one_trade_df['volume_x'].iloc[0]
    
    else:
        y_profit = -one_trade_df['price_y'].diff()*one_trade_df['volume_y'].iloc[0]
        x_profit = one_trade_df['price_x'].diff()*one_trade_df['volume_x'].iloc[0]
    start_spread = one_trade_df['price_y'].iloc[0]*one_trade_df['volume_y'].iloc[0]-one_trade_df['price_x'].iloc[0]*one_trade_df['volume_x'].iloc[0]
    close_spread = one_trade_df['price_y'].iloc[1]*one_trade_df['volume_y'].iloc[1]-one_trade_df['price_x'].iloc[1]*one_trade_df['volume_x'].iloc[0]
    start_datetime = one_trade_df['datetime'].iloc[0]
    end_datetime = one_trade_df['datetime'].iloc[1]
    direction = one_trade_df['direction_y'].iloc[0]
    trades_analyze_df = trades_analyze_df.append({'open_date':start_datetime,'close_date':end_datetime,
                                                  'direction':direction,
                                                  'open_spread':start_spread,
                                                  'close_spread':close_spread,
                                                  'y_profit':y_profit,
                                                  'x_profit':x_profit},ignore_index=True)
    


trades_analyze_df['total_profit'] = (trades_analyze_df['y_profit']+trades_analyze_df['x_profit'])*np.where(trades_analyze_df['direction']=='多',1,-1)

In [ ]:
trades_analyze_df

In [ ]:
combine_trade_df

In [ ]:
spread_datetime_list = engine.strategy.record_array
boll_up_datetime_list = engine.strategy.record_boll_up
boll_down_datetime_list = engine.strategy.record_boll_down


datetime_list = [ spread_datetime[0] for spread_datetime in spread_datetime_list]
spread_list = [ spread_datetime[1] for spread_datetime in spread_datetime_list]
diff_list = [ spread_datetime[2] for spread_datetime in spread_datetime_list]

datetime_boll_up_list = [ boll_up_datetime[0] for boll_up_datetime in boll_up_datetime_list]
boll_up_list = [  boll_up_datetime[1] for boll_up_datetime in boll_up_datetime_list]

datetime_boll_down_list = [ boll_down_datetime[0] for boll_down_datetime in boll_down_datetime_list]
boll_down_list = [  boll_down_datetime[1] for boll_down_datetime in boll_down_datetime_list]

In [ ]:
value_df = pd.DataFrame({'datetime':datetime_list,'residual':spread_list,'diff':diff_list})
boll_df = pd.DataFrame({'datetime':datetime_boll_up_list,'boll_up':boll_up_list,'boll_down':boll_down_list})
# value_df.set_index('datetime',inplace=True)

In [ ]:
combine_df = pd.merge(left=value_df,right=boll_df)
combine_df = combine_df.iloc[10*240:]

In [ ]:
combine_df.loc[combine_df['residual']>combine_df['boll_up']]

In [ ]:
combine_df.loc[combine_df['residual']<combine_df['boll_down']]

In [ ]:
bar_1 = Line()

start = 0
end = 20000
# 以时间作为x轴
bar_1.add_xaxis(combine_df['datetime'].tolist()[start:end],)
# y轴数据字符串表示，y轴数据list
bar_1.add_yaxis('OLS_residual',combine_df['residual'].tolist()[start:end])
bar_1.add_yaxis('boll_up',combine_df['boll_up'].tolist()[start:end])
bar_1.add_yaxis('boll_down',combine_df['boll_down'].tolist()[start:end])
bar_1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  
bar_1.render_notebook()

In [ ]:
bar_1 = Line()

start = 20000
end = 40000
# 以时间作为x轴
bar_1.add_xaxis(combine_df['datetime'].tolist()[start:end],)
# y轴数据字符串表示，y轴数据list
bar_1.add_yaxis('OLS_residual',combine_df['residual'].tolist()[start:end])
bar_1.add_yaxis('boll_up',combine_df['boll_up'].tolist()[start:end])
bar_1.add_yaxis('boll_down',combine_df['boll_down'].tolist()[start:end])
bar_1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  
bar_1.render_notebook()

In [ ]:
bar_1 = Line()

start = 40000
end = 60000
# 以时间作为x轴
bar_1.add_xaxis(combine_df['datetime'].tolist()[start:end],)
# y轴数据字符串表示，y轴数据list
bar_1.add_yaxis('OLS_residual',combine_df['residual'].tolist()[start:end])
bar_1.add_yaxis('boll_up',combine_df['boll_up'].tolist()[start:end])
bar_1.add_yaxis('boll_down',combine_df['boll_down'].tolist()[start:end])
bar_1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  
bar_1.render_notebook()

In [ ]:
bar_1 = Line()

start = 2000
end = 2300
# 以时间作为x轴
bar_1.add_xaxis(combine_df['datetime'].tolist()[start:end],)
# y轴数据字符串表示，y轴数据list
bar_1.add_yaxis('diff',combine_df['diff'].tolist()[start:end])
bar_1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  
bar_1.render_notebook()


In [ ]:
combine_df['diff'].iloc[2500:3000].plot()

In [ ]:
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm

In [ ]:
X = combine_df['residual']

y = combine_df['diff']

result = sm.OLS(y,X).fit()


In [ ]:
result.params

In [ ]:
engine.__dict__